# Phân Tích Toàn Diện FaceNet Model

Notebook này thực hiện phân tích và trực quan hóa toàn bộ kết quả training và đánh giá của mô hình FaceNet.

## Nội dung:
1. **Training Analysis**: Loss curves, Accuracy curves, Learning trajectory
2. **Evaluation Metrics**: Top-1/Top-5 Accuracy, AUC, EER
3. **ROC Curve Analysis**: Đường cong ROC chi tiết
4. **Threshold Analysis**: Phân tích ngưỡng quyết định
5. **Confusion Matrix**: Ma trận nhầm lẫn
6. **Model Configuration**: Cấu hình huấn luyện
7. **Báo cáo tổng hợp**: Đánh giá toàn diện

## 1. Setup và Import

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from IPython.display import display, Markdown, Image
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

LOGS_DIR = Path('../logs/facenet')
print(f"Logs directory: {LOGS_DIR.absolute()}")
print(f"Các file có sẵn:")
for f in sorted(LOGS_DIR.glob('*')):
    print(f"  - {f.name} ({f.stat().st_size / 1024:.1f} KB)")

## 2. Load Dữ Liệu

In [ ]:
with open(LOGS_DIR / 'training_history.json', 'r') as f:
    training_data = json.load(f)

with open(LOGS_DIR / 'facenet_evaluation_report.json', 'r') as f:
    eval_report = json.load(f)

with open(LOGS_DIR / 'facenet_evaluation_data.json', 'r') as f:
    eval_data = json.load(f)

predictions_df = pd.read_csv(LOGS_DIR / 'facenet_predictions.csv')

metrics = eval_report['metrics']
metadata = eval_data['metadata']

print("Data loaded successfully!")
print(f"Training epochs: {len(training_data['train_loss'])}")
print(f"Number of identities: {metadata['num_identities']}")
print(f"Number of samples: {metadata['num_samples']}")
print(f"Embedding size: {metadata['embedding_size']}")

---
# PHẦN 1: PHÂN TÍCH QUÁ TRÌNH TRAINING
---

## 3. Training Curves - Loss và Accuracy

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

epochs = list(range(1, len(training_data['train_loss']) + 1))

ax1 = axes[0, 0]
ax1.plot(epochs, training_data['train_loss'], 'b-', lw=2, label='Train Loss', alpha=0.8)
ax1.plot(epochs, training_data['val_loss'], 'r-', lw=2, label='Val Loss', alpha=0.8)
ax1.fill_between(epochs, training_data['train_loss'], training_data['val_loss'], alpha=0.2, color='gray')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Triplet Loss Over Training', fontweight='bold')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

ax2 = axes[0, 1]
train_acc = [x * 100 for x in training_data['train_acc']]
val_acc = [x * 100 for x in training_data['val_acc']]
ax2.plot(epochs, train_acc, 'b-', lw=2, label='Train Acc', alpha=0.8)
ax2.plot(epochs, val_acc, 'r-', lw=2, label='Val Acc', alpha=0.8)
ax2.fill_between(epochs, train_acc, val_acc, alpha=0.2, color='gray')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.set_title('Accuracy Over Training', fontweight='bold')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)

ax3 = axes[1, 0]
gap = np.array(train_acc) - np.array(val_acc)
colors = ['green' if g < 5 else 'orange' if g < 10 else 'red' for g in gap]
ax3.bar(epochs, gap, color=colors, alpha=0.7)
ax3.axhline(y=5, color='orange', linestyle='--', label='Warning threshold (5%)')
ax3.axhline(y=10, color='red', linestyle='--', label='Overfitting threshold (10%)')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Gap (%)')
ax3.set_title('Generalization Gap (Train - Val)', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
ax4.plot(epochs, training_data['lr'], 'g-', lw=2, marker='o', markersize=3)
ax4.set_xlabel('Epoch')
ax4.set_ylabel('Learning Rate')
ax4.set_title('Learning Rate Schedule', fontweight='bold')
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(LOGS_DIR / 'training_curves_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTraining Summary:")
print(f"  Final Train Loss: {training_data['train_loss'][-1]:.4f}")
print(f"  Final Val Loss: {training_data['val_loss'][-1]:.4f}")
print(f"  Final Train Acc: {train_acc[-1]:.2f}%")
print(f"  Final Val Acc: {val_acc[-1]:.2f}%")
print(f"  Final Gap: {gap[-1]:.2f}%")

## 4. Loss Dynamics Analysis

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

ax1 = axes[0]
train_loss_diff = np.diff(training_data['train_loss'])
val_loss_diff = np.diff(training_data['val_loss'])
ax1.plot(epochs[1:], train_loss_diff, 'b-', lw=1.5, label='Train', alpha=0.7)
ax1.plot(epochs[1:], val_loss_diff, 'r-', lw=1.5, label='Val', alpha=0.7)
ax1.axhline(y=0, color='black', linestyle='--', alpha=0.5)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss Change')
ax1.set_title('Loss Change per Epoch', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
window = 5
train_loss_smooth = np.convolve(training_data['train_loss'], np.ones(window)/window, mode='valid')
val_loss_smooth = np.convolve(training_data['val_loss'], np.ones(window)/window, mode='valid')
ax2.plot(range(1, len(train_loss_smooth)+1), train_loss_smooth, 'b-', lw=2, label='Train (smoothed)')
ax2.plot(range(1, len(val_loss_smooth)+1), val_loss_smooth, 'r-', lw=2, label='Val (smoothed)')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.set_title(f'Smoothed Loss (window={window})', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

ax3 = axes[2]
loss_ratio = np.array(training_data['val_loss']) / np.array(training_data['train_loss'])
colors = ['green' if r < 1.2 else 'orange' if r < 1.5 else 'red' for r in loss_ratio]
ax3.scatter(epochs, loss_ratio, c=colors, s=50, alpha=0.7)
ax3.axhline(y=1, color='gray', linestyle='--', alpha=0.5)
ax3.axhline(y=1.2, color='orange', linestyle='--', label='Warning (1.2)')
ax3.axhline(y=1.5, color='red', linestyle='--', label='Overfitting (1.5)')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Val/Train Loss Ratio')
ax3.set_title('Loss Ratio Analysis', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(LOGS_DIR / 'loss_dynamics.png', dpi=150, bbox_inches='tight')
plt.show()

---
# PHẦN 2: ĐÁNH GIÁ MÔ HÌNH
---

## 5. Evaluation Metrics Summary

In [ ]:
print('='*70)
print('                    FACENET EVALUATION SUMMARY')
print('='*70)
print(f'\n  Model: {metadata["model"]}')
print(f'  Embedding Size: {metadata["embedding_size"]}')
print(f'  Number of Identities: {metadata["num_identities"]}')
print(f'  Number of Test Samples: {metadata["num_samples"]}')
print(f'  Evaluation Timestamp: {metadata["timestamp"]}')
print('\n' + '-'*70)
print('  RECOGNITION METRICS:')
print('-'*70)
print(f'  Top-1 Accuracy: {metrics["top1_accuracy"]:.2f}%')
print(f'  Top-5 Accuracy: {metrics["top5_accuracy"]:.2f}%')
print('\n' + '-'*70)
print('  VERIFICATION METRICS:')
print('-'*70)
print(f'  AUC-ROC: {metrics["auc"]:.4f}')
print(f'  EER (Equal Error Rate): {metrics["eer"]*100:.2f}%')
print('='*70)

## 6. Metrics Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

ax1 = axes[0]
metric_names = ['Top-1 Acc', 'Top-5 Acc', 'AUC×100']
metric_values = [metrics['top1_accuracy'], metrics['top5_accuracy'], metrics['auc'] * 100]
colors = ['#3498db', '#2ecc71', '#9b59b6']
bars = ax1.bar(metric_names, metric_values, color=colors, edgecolor='black', linewidth=1.5)
ax1.set_ylim([0, 105])
ax1.set_ylabel('Value (%)')
ax1.set_title('Recognition & Verification Metrics', fontweight='bold')
for bar, val in zip(bars, metric_values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, f'{val:.1f}%', 
             ha='center', va='bottom', fontweight='bold', fontsize=11)
ax1.grid(axis='y', alpha=0.3)

ax2 = axes[1]
sizes = [100 - metrics['eer']*100, metrics['eer']*100]
labels = ['Correct', 'EER']
colors_pie = ['#2ecc71', '#e74c3c']
explode = (0.02, 0.05)
ax2.pie(sizes, explode=explode, labels=labels, colors=colors_pie, autopct='%1.2f%%',
        shadow=True, startangle=90, textprops={'fontsize': 11})
ax2.set_title('Equal Error Rate Analysis', fontweight='bold')

ax3 = axes[2]
categories = ['Top-1', 'Top-5', 'AUC', '1-EER']
values = [
    metrics['top1_accuracy'] / 100,
    metrics['top5_accuracy'] / 100,
    metrics['auc'],
    1 - metrics['eer']
]
angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False).tolist()
values_radar = values + [values[0]]
angles += angles[:1]
ax3 = plt.subplot(133, polar=True)
ax3.plot(angles, values_radar, 'o-', linewidth=2, color='#3498db')
ax3.fill(angles, values_radar, alpha=0.25, color='#3498db')
ax3.set_xticks(angles[:-1])
ax3.set_xticklabels(categories)
ax3.set_ylim(0, 1)
ax3.set_title('Overall Performance Radar', fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig(LOGS_DIR / 'metrics_comprehensive.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. ROC Curve Analysis

In [ ]:
roc_img_path = LOGS_DIR / 'facenet_roc_curve.png'
if roc_img_path.exists():
    print("ROC Curve:")
    display(Image(filename=str(roc_img_path), width=700))
    print(f"\nAUC Score: {metrics['auc']:.4f}")
    print(f"Interpretation: ", end='')
    if metrics['auc'] >= 0.95:
        print("Excellent - Model có khả năng phân biệt rất tốt")
    elif metrics['auc'] >= 0.90:
        print("Good - Model có khả năng phân biệt tốt")
    elif metrics['auc'] >= 0.80:
        print("Fair - Model có khả năng phân biệt khá")
    else:
        print("Poor - Cần cải thiện model")
else:
    print("ROC curve image not found.")

## 8. Threshold Analysis

In [ ]:
threshold_img_path = LOGS_DIR / 'facenet_threshold_analysis.png'
if threshold_img_path.exists():
    print("Threshold Analysis:")
    display(Image(filename=str(threshold_img_path), width=700))
    print(f"\nOptimal Threshold Analysis:")
    print(f"  EER (Equal Error Rate): {metrics['eer']*100:.2f}%")
    print(f"  EER là điểm mà FAR = FRR")
else:
    print("Threshold analysis image not found.")

## 9. Confusion Matrix

In [ ]:
cm_img_path = LOGS_DIR / 'facenet_confusion_matrix.png'
if cm_img_path.exists():
    print("Confusion Matrix:")
    display(Image(filename=str(cm_img_path), width=700))
else:
    print("Confusion matrix image not found.")

## 10. Predictions Analysis

In [ ]:
print(f"Predictions DataFrame Shape: {predictions_df.shape}")
print(f"\nColumns: {list(predictions_df.columns)}")
print(f"\nFirst 10 rows:")
display(predictions_df.head(10))

if 'correct' in predictions_df.columns:
    correct_count = predictions_df['correct'].sum()
    total_count = len(predictions_df)
    print(f"\nCorrect Predictions: {correct_count}/{total_count} ({correct_count/total_count*100:.2f}%)")

In [ ]:
if 'similarity' in predictions_df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1 = axes[0]
    ax1.hist(predictions_df['similarity'], bins=50, color='#3498db', edgecolor='black', alpha=0.7)
    ax1.set_xlabel('Similarity Score')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Distribution of Similarity Scores', fontweight='bold')
    ax1.axvline(x=predictions_df['similarity'].mean(), color='red', linestyle='--', 
                label=f'Mean: {predictions_df["similarity"].mean():.3f}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    if 'correct' in predictions_df.columns:
        ax2 = axes[1]
        correct_sims = predictions_df[predictions_df['correct'] == True]['similarity']
        incorrect_sims = predictions_df[predictions_df['correct'] == False]['similarity']
        ax2.hist(correct_sims, bins=30, color='green', alpha=0.6, label=f'Correct (n={len(correct_sims)})')
        ax2.hist(incorrect_sims, bins=30, color='red', alpha=0.6, label=f'Incorrect (n={len(incorrect_sims)})')
        ax2.set_xlabel('Similarity Score')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Similarity by Prediction Correctness', fontweight='bold')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(LOGS_DIR / 'similarity_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()

---
# PHẦN 3: CẤU HÌNH VÀ BÁO CÁO
---

## 11. Model Configuration

In [ ]:
print('='*70)
print('                    MODEL CONFIGURATION')
print('='*70)
print(f'\n  Model Architecture: FaceNet (Inception-ResNet v1)')
print(f'  Embedding Size: {metadata["embedding_size"]}')
print(f'  Loss Function: Triplet Loss with Semi-Hard Negative Mining')
print(f'\n  Training Configuration:')
print(f'    - Epochs: {len(training_data["train_loss"])}')
print(f'    - Initial Learning Rate: {training_data["lr"][0]:.6f}')
print(f'    - Final Learning Rate: {training_data["lr"][-1]:.6f}')
print(f'\n  Dataset:')
print(f'    - Number of Identities: {metadata["num_identities"]}')
print(f'    - Number of Test Samples: {metadata["num_samples"]}')
print('='*70)

## 12. Comprehensive Summary Report

In [ ]:
summary = {
    'model': 'FaceNet',
    'embedding_size': metadata['embedding_size'],
    'epochs': len(training_data['train_loss']),
    'num_identities': metadata['num_identities'],
    'num_samples': metadata['num_samples'],
    'final_train_loss': training_data['train_loss'][-1],
    'final_val_loss': training_data['val_loss'][-1],
    'final_train_acc': training_data['train_acc'][-1] * 100,
    'final_val_acc': training_data['val_acc'][-1] * 100,
    'top1_accuracy': metrics['top1_accuracy'],
    'top5_accuracy': metrics['top5_accuracy'],
    'auc': metrics['auc'],
    'eer': metrics['eer'],
    'generalization_gap': (training_data['train_acc'][-1] - training_data['val_acc'][-1]) * 100
}

with open(LOGS_DIR / 'comprehensive_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print('='*70)
print('                 COMPREHENSIVE SUMMARY REPORT')
print('='*70)
print(json.dumps(summary, indent=2))
print('='*70)
print(f'\nSummary saved to: {LOGS_DIR / "comprehensive_summary.json"}')

## 13. Performance Assessment

In [ ]:
print('='*70)
print('                    PERFORMANCE ASSESSMENT')
print('='*70)

print('\n1. RECOGNITION PERFORMANCE:')
if metrics['top1_accuracy'] >= 90:
    print(f'   [EXCELLENT] Top-1 Accuracy: {metrics["top1_accuracy"]:.2f}%')
elif metrics['top1_accuracy'] >= 80:
    print(f'   [GOOD] Top-1 Accuracy: {metrics["top1_accuracy"]:.2f}%')
else:
    print(f'   [NEEDS IMPROVEMENT] Top-1 Accuracy: {metrics["top1_accuracy"]:.2f}%')

print('\n2. VERIFICATION PERFORMANCE:')
if metrics['auc'] >= 0.95:
    print(f'   [EXCELLENT] AUC: {metrics["auc"]:.4f}')
elif metrics['auc'] >= 0.90:
    print(f'   [GOOD] AUC: {metrics["auc"]:.4f}')
else:
    print(f'   [NEEDS IMPROVEMENT] AUC: {metrics["auc"]:.4f}')

print('\n3. GENERALIZATION:')
gap = summary['generalization_gap']
if gap < 5:
    print(f'   [EXCELLENT] Generalization Gap: {gap:.2f}% - Model generalizes well')
elif gap < 10:
    print(f'   [GOOD] Generalization Gap: {gap:.2f}% - Slight overfitting')
else:
    print(f'   [WARNING] Generalization Gap: {gap:.2f}% - Significant overfitting')

print('\n4. OVERALL ASSESSMENT:')
overall_score = (metrics['top1_accuracy']/100 + metrics['auc'] + (1 - metrics['eer'])) / 3 * 100
print(f'   Overall Score: {overall_score:.2f}/100')
print('='*70)